In [49]:
# 데이터 처리 라이브러리

import pandas as pd
import numpy as np

# 머신러닝 라이브러리
from sklearn.svm import SVC # 서포트 vector
from sklearn.neighbors import KNeighborsClassifier # k neighbors

# 데이터 전처리 라이브러리
from sklearn.model_selection import train_test_split

# classification metrix
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

In [4]:
# 데이터 불러오기

csData = pd.read_csv("../../dataset/customer.csv")
csData.label.drop_duplicates()

0     normal
1    diamond
4        vip
Name: label, dtype: object

### 1. 타입 통합 / 숫자형 컬럼 추가

In [8]:
# skip
csData.dtypes

balance     int64
stock       int64
label      object
dtype: object

In [10]:
csData = csData.astype({"balance" : "int", "stock" : "int", "label" : "object"})

### 2. 특성 선정 / 데이터 분리

#### 2-1. 특성 선정

In [19]:
# label_new를 통해 features를 찾기 위해 해당 값을 설정
labelColumn = "label_new"
corrStd = 0.5

In [13]:
# label_encoder를 이용해서 자동으로 값을 매기거나, 딕셔너리를 통해 아랴처럼 할 수 있다.
# 단, 전자의 경우 내가 원하는 값으로 밉핑하기 제한됨.

labelMap = {"normal" : 0, "diamond" : 1, "vip" : 2}

In [17]:
csData[labelColumn] = csData.label.map(labelMap)

In [21]:
# 상관계수 생성 함수는 숫자형 컬럼에 대해서만 상관계수를 계산한다.
corrDF = csData.corr()

In [30]:
features = list(corrDF.loc[(abs(corrDF[labelColumn]) > corrStd) & (corrDF[labelColumn] != 1)].index)

In [56]:
features

['balance', 'stock']

In [27]:
set(labelColumn)

{'_', 'a', 'b', 'e', 'l', 'n', 'w'}

In [26]:
# set는 차집합연산을 가능하게 해준다.
set(features)

{'LABEL_NEW', 'balance', 'label', 'label_new', 'stock'}

#### 2-2. 데이터 분리 

In [42]:
# feature값 확인 후 해당 라벨값 초기화
labelColumn = "label"
features = list(corrDF.loc[(abs(corrDF[labelColumn]) > corrStd) & (corrDF[labelColumn] != 1)].index)

traningDataFeatures, \
testDataFeatures, \
trainingDataLabel, \
testDataLabel = \
train_test_split(csData.loc[:, features],
                csData.loc[:, labelColumn],
                random_state = 1,
                test_size = 0.2)

In [37]:
print(traningDataFeatures)
print(testDataFeatures)
print(trainingDataLabel)
print(testDataLabel)

         balance     stock
7013   252000000  17500000
7766   290000000  18000000
5910   684000000  35333333
7020   628000000  30000000
16952  532000000  50666666
...          ...       ...
10955  792000000  28000000
17289  780000000  30666666
5192   306000000  22500000
12172  708000000  74000000
235    716000000  36666666

[16000 rows x 2 columns]
         balance     stock
11456  744000000  38000000
16528  724000000  32000000
3253   704000000  27333333
18614  240000000  30500000
1544   258000000  28500000
...          ...       ...
6375   640000000  48666666
14837  652000000  26000000
3931   564000000  45333333
18266  528000000  46000000
12028  696000000  36666666

[4000 rows x 2 columns]
7013     0
7766     0
5910     1
7020     1
16952    1
        ..
10955    1
17289    1
5192     0
12172    2
235      1
Name: label, Length: 16000, dtype: int64
11456    1
16528    1
3253     1
18614    0
1544     0
        ..
6375     1
14837    1
3931     1
18266    1
12028    1
Name: label, Lengt

### 3. 모델 선언 / 학습

In [50]:
model_svm = SVC(random_state = 5)

In [51]:
model_svm.fit(traningDataFeatures, trainingDataLabel)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=5, shrinking=True, tol=0.001,
    verbose=False)

### 4. 예측 

In [52]:
predictSvm = model_svm.predict(testDataFeatures)

### 5. 데이터 정리

In [53]:
testDataAll = csData.loc[testDataLabel.index]

In [54]:
testDataAll["predict_svm"] = predictSvm

### 6. 정확도 검증

In [55]:
accuracy = accuracy_score(y_true = testDataAll.label,
              y_pred = testDataAll.predict_svm)

In [ ]:
testDataAll.to_clipboard("d:/svc_res.csv", index = False)

##### regression 의 경우 답지가 불가산이나,
##### classification 의 경우 답지가 얼마 없어 정확도가 높다.